<a href="https://colab.research.google.com/github/zlximon/Tensorflow_DL/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#downloading helper functions 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2022-07-06 18:38:37--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-06 18:38:37 (74.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys



#get text dataset

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-07-06 18:38:40--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 216.58.203.240, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-06 18:38:40 (148 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



  #visualizing text dataset

In [4]:
import pandas as pd 

In [5]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()
# 1 is for disaster 

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
len(train_df),len(test_df)

(7613, 3263)

In [10]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
@SonofBaldwin and he's the current Nova in the bookslast I checked..he was tied into the books in 2011 after Rider died during Annihilation

---

Target: 1 (real disaster)
Text:
Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all

---

Target: 0 (not real disaster)
Text:
Do you feel engulfed with low self-image? Take the quiz: http://t.co/YzDmouXQBO http://t.co/PeXfgawrG1

---

Target: 1 (real disaster)
Text:
Investigators shift focus to cause of fatal Waimate fire http://t.co/GoeTJGIhOp

---

Target: 0 (not real disaster)
Text:
First Time Playing Hearthstone on PC Thoughts http://t.co/aBoLxMH1vy

---



#split data into training and validation  sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),train_df_shuffled["target"],test_size=0.1,
                                                                       random_state=42)

In [13]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [14]:
train_sentences[:10],train_labels[:10 ]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), 5921    0


Embedding : create a matrix of features vector for each token 

#text vectorization ( tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
text_vectorizer=TextVectorization(max_tokens=None# how many word in vocalubary  and None means there is no cap 
                                  ,standardize="lower_and_strip_punctuation",# turn all words to lower case and strip 
                                  split="whitespace",ngrams=None,output_mode="int",output_sequence_length=None,# how long do we want sequences be 
                                 )# pad_to_max_tokens=True)#add zeros into end of short lenght 

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None # how long do you want your sequences to be?
                                    )

In [18]:
#find the average numbers of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [19]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# fit the text vectorization to the training text
text_vectorizer.adapt(train_sentences)


In [21]:
#create a sample 
sample="There is a flood in my street"
text_vectorizer([sample])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
#choose a random sentence from training dataset
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Heavy smoke pouring out of buildings on fire in Port Coquitlam http://t.co/GeqkdaO4cV http://t.co/Dg0bGzeCgM        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 890,  271, 9652,   36,    6,   95,   11,   42,    4, 2342,    1,
           1,    1,    0,    0]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating Embedding layer 

Embedding : turns positives integers into dense vectors of fixed size
parameters for our embedding

1-input_dim - The size of the vocabulary

2-output_dim - The size of the output embedding vector, for example, a value of 100 outputs a feature vector of size 100 for each word.

3-embeddings_initializer - How to initialize the embeddings matrix, default is "uniform" which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.

4- input_length - Length of sequences being passed to embedding layer.


In [24]:
from tensorflow.keras import layers 
embedding =layers.Embedding(input_dim=max_vocab_length,output_dim=128,input_length=max_length)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
in fact if y'all could tag like small creeping or self inflicted injuries of the skin as derma (with brackets) that would be nice      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01480243,  0.01663173,  0.02885399, ...,  0.02437737,
          0.04945684,  0.03815782],
        [ 0.02999461, -0.04736364,  0.03838307, ..., -0.04001445,
          0.02748898, -0.00498121],
        [-0.01018405,  0.03411936,  0.00161924, ...,  0.04496739,
          0.00871556,  0.03832834],
        ...,
        [ 0.0352551 ,  0.00317834,  0.03692337, ...,  0.03146312,
          0.00639443, -0.00301248],
        [ 0.04696215, -0.00450442,  0.03473593, ..., -0.0356328 ,
          0.00154173,  0.02358003],
        [-0.01510154, -0.02939985, -0.02716184, ...,  0.01393772,
         -0.0235989 ,  0.03750192]]], dtype=float32)>

In [26]:
sample_embed[0][0],sample_embed[0][0].shape,random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-1.48024336e-02,  1.66317336e-02,  2.88539864e-02,  1.90145485e-02,
        -3.61132398e-02,  1.11587048e-02, -2.25618612e-02,  3.30455415e-02,
        -3.38518396e-02,  1.46694817e-02, -2.93344148e-02,  5.01482561e-03,
        -2.41608266e-02, -1.48290284e-02, -4.17344458e-02, -3.78808975e-02,
        -1.19680539e-02,  3.78169678e-02,  4.76148836e-02, -8.36988539e-03,
         2.23367102e-02,  2.05256604e-02, -4.16353233e-02, -2.44351625e-02,
        -3.83634195e-02,  1.42009966e-02, -1.74878016e-02,  1.64399631e-02,
        -2.91073080e-02,  3.35769765e-02,  4.02312353e-03,  1.52029134e-02,
        -2.59724613e-02, -4.95861061e-02, -1.54882446e-02,  1.92740075e-02,
         1.41524412e-02,  4.33862470e-02,  3.22220661e-02, -1.67320743e-02,
         2.81767137e-02, -1.71244144e-02,  1.71743333e-04,  2.69887932e-02,
         5.01612574e-03,  4.38094139e-05,  2.17488445e-02,  4.72317375e-02,
        -3.92196774e-02,  1.92966722e-0

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [28]:
model0=Pipeline([
("tfidf",TfidfVectorizer()),#convert words to numbers using tfdif
("clf",MultinomialNB())#model the text
])
#fit pipeline to train data
model0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
baseline_score=model0.score(val_sentences,val_labels)# evaluate 
baseline_score # accuracy 

0.7926509186351706

In [30]:
baseline_pred=model0.predict(val_sentences)
baseline_pred[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

#creating function to track and evaluate model 

In [31]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [32]:
baseline_result=calculate_results(y_true=val_labels,y_pred=baseline_pred)
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#model 1 (Dense model)

In [33]:
from helper_functions import create_tensorboard_callback


In [34]:
SAVE_DIR = "model_logs"


In [35]:
from tensorflow.keras import layers
inputs=(layers.Input(shape=(1,),dtype=tf.string))
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
#x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model1=tf.keras.Model(inputs,outputs,name="model1")

In [36]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [37]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [38]:
#compile model
model1.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [39]:
#fit the model
model1_history=model1.fit(x=train_sentences,y=train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220706-183848
Epoch 1/5
215/215 [==============================] - 8s 9ms/step - loss: 0.6122 - accuracy: 0.6898 - val_loss: 0.5382 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4416 - accuracy: 0.8197 - val_loss: 0.4701 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3471 - accuracy: 0.8615 - val_loss: 0.4561 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2846 - accuracy: 0.8905 - val_loss: 0.4659 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2377 - accuracy: 0.9101 - val_loss: 0.4759 - val_accuracy: 0.7835


In [40]:
model1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4759 - accuracy: 0.7835


[0.47589927911758423, 0.7834645509719849]

In [41]:
model1_pred_probs=model1.predict(val_sentences)
model1_pred_probs.shape

(762, 1)

In [42]:
model1_pred_probs[0]

array([0.36966968], dtype=float32)

In [43]:
#Convert model prediction probabilites to label format 
model1_preds=tf.squeeze(tf.round(model1_pred_probs))
model1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [44]:
model1_result=calculate_results(y_true=val_labels,y_pred=model1_preds)
model1_result


{'accuracy': 78.34645669291339,
 'f1': 0.7813141680786296,
 'precision': 0.785851650078301,
 'recall': 0.7834645669291339}

In [45]:
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#visualazing learned embeddings

In [46]:
#get the vocabulary from  the vectorization layer
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [47]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [48]:
embed_weights=model1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [49]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [50]:
# Let's download the saved embeddings locally

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#recurrent neural Networks (RNN)

#model 2 LSTM 
long short term momory 

In [51]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
#x=layers.LSTM(64,return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) 
x=layers.Dense(64,activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model2=tf.keras.Model(inputs,outputs,name="model2")

In [52]:
model2.summary()

Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                            

In [53]:
#compile the model
model2.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [54]:
#fit the model
model2_history=model2.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels), callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220706-183905
Epoch 1/5
215/215 [==============================] - 6s 8ms/step - loss: 0.2160 - accuracy: 0.9223 - val_loss: 0.5444 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1555 - accuracy: 0.9435 - val_loss: 0.5679 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1272 - accuracy: 0.9527 - val_loss: 0.6602 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1090 - accuracy: 0.9580 - val_loss: 0.9103 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0849 - accuracy: 0.9677 - val_loss: 0.8405 - val_accuracy: 0.7822


In [55]:
#make prediction 
model2_pred_probs=model2.predict(val_sentences)
model2_pred_probs[:10]

array([[1.0489121e-02],
       [8.0225986e-01],
       [9.9981767e-01],
       [9.7844273e-02],
       [2.0182769e-04],
       [9.9266362e-01],
       [5.9961718e-01],
       [9.9995995e-01],
       [9.9981254e-01],
       [4.6099186e-01]], dtype=float32)

In [56]:
# Round out pred probs and reduce to 1-dimensional array

model_2_preds = tf.squeeze(tf.round(model2_pred_probs))


In [57]:
# Calculate our LSTM model's results

model_2_preds = tf.squeeze(tf.round(model_2_preds))


In [58]:
# Calculate our model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 78.21522309711287,
 'f1': 0.7787825179570674,
 'precision': 0.7877268665884849,
 'recall': 0.7821522309711286}

#model 3 - GRU model

In [75]:
#build RNN model using GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x=layers.LSTM(64,return_sequences=True)(x)
#x=layers.GRU(64,)(x)
#x=layers.Dense(64,activation="relu")(x)
x=layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


In [72]:
#model 3 summary with out GlobalAverage pooling
model3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 dense_17 (Dense)            (None, 15, 64)            8256      
                                                                 
 dense_18 (Dense)            (None, 15, 1)             65        
                                                                 
Total params: 1,288,321
Trainable params: 1,288,321
Non-trainable params: 0
_____________________________________________

In [76]:
#model 3 summary with our GlobalAverage pooling
model3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_3   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_21 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [78]:
#compile the model
model3.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [81]:
#fit the model
model3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220706-185647
Epoch 1/5
215/215 [==============================] - 2s 8ms/step - loss: 0.5154 - accuracy: 0.8406 - val_loss: 0.4991 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3201 - accuracy: 0.9002 - val_loss: 0.4586 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2418 - accuracy: 0.9251 - val_loss: 0.4633 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1976 - accuracy: 0.9371 - val_loss: 0.4847 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1678 - accuracy: 0.9482 - val_loss: 0.5057 - val_accuracy: 0.7835


In [85]:
model3_pred_probs=model3.predict(val_sentences)
model3_pred_probs[:10]

array([[0.3789759 ],
       [0.7509018 ],
       [0.996684  ],
       [0.12125313],
       [0.03652029],
       [0.9423841 ],
       [0.8687943 ],
       [0.99725986],
       [0.9878947 ],
       [0.35950977]], dtype=float32)

In [86]:
model3_preds=tf.squeeze(tf.round(model3_pred_probs))
model3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [89]:
model3_result=calculate_results(y_true=val_labels,y_pred=model3_preds)
model3_result

{'accuracy': 78.34645669291339,
 'f1': 0.7810537461695031,
 'precision': 0.7864953185428455,
 'recall': 0.7834645669291339}